In [ ]:
from pyspark.streaming import StreamingContext
import sys

In [ ]:
outputPathPrefix="resSparkStreamingWinodws"

In [ ]:
# Create a Spark Streaming Context object
ssc = StreamingContext(sc, 5)

In [ ]:
# Set the checkpoint folder (it is needed by some window transformations)
ssc.checkpoint("checkpointfolder")

In [ ]:
# Create a (Receiver) DStream that will connect to localhost:9999
lines = ssc.socketTextStream("localhost", 9999)

In [ ]:
# Apply the "standard" transformations to perform the word count task
# However, the "returned" RDDs are DStream RDDs
words = lines.flatMap(lambda line: line.split(" "))

In [ ]:
wordsOnes = words.map(lambda word: (word, 1))

In [ ]:
# Define the function that is used to update the state of a key at a time
def updateFunction(newValues, currentCount):
    if currentCount is None:
        currentCount = 0
        
    # Sum the new values to the previous state for the current key
    return sum(newValues, currentCount)  

In [ ]:
# DStream made of cumulative counts for each key that get updated in every batch
totalWordsCounts = wordsOnes.updateStateByKey(updateFunction)

In [ ]:
totalWordsCounts.pprint()

In [ ]:
totalWordsCounts.saveAsTextFiles(outputPathPrefix, "")

In [ ]:
#Start the computation
ssc.start()

In [ ]:
# Run this application for 90 seconds
ssc.awaitTerminationOrTimeout(90)
ssc.stop(stopSparkContext=False)